# Multirun experiment pipeline 0.3.0 (CN-Wheat)

In [1]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing local packages from src

%reload_ext autoreload
%autoreload 2 

### Loading the datasets

These datasets were collected and converted in the WheatFspm repository.

There are three simulations made available in the WheatFspm repository that are useable for RC experiments: NEMA H0, H3 and H15.

We can try using these datasets in two different ways:

1. Treat every dataset as a separate plant, training a readout for each simulation run.
2. Concatenating the three datasets as observed behavior of a single plant.

In [2]:
from src.model.rc_dataset import ExperimentDataset


DATASET_PATH = 'datasets/hydroshoot_large_trimmed.csv'

dataset = ExperimentDataset(csv_path=DATASET_PATH)

### Model definition

In [3]:
DATASET_ID = 'HydroShoot_large'
STATE_SIZE=16

In [4]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold
from src.learning.scorers import nmse_scorer
from pipeline_hydroshoot import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform


readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])

model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

datasets = (dataset,)
run_ids = dataset.get_run_ids()
np.random.seed(42)
state_ids = np.random.choice(dataset.state_size(), size=16, replace=False)

shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24, run_ids=run_ids, days_between_runs=1),
  'time': TimeGenerator(day_length=24, run_ids=run_ids),

  # Data transformation
  'transforms': [
    WarmupTransform(warmup_days=4, day_length=24),
    DirectTransform()
  ],
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': GroupKFold(n_splits=5),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

### Generating model fit pipelines

In [5]:
from model_config_hydroshoot import (
  targets, 
  state_variables,
  baseline_reservoirs,
  heterogeneous_reservoirs, 
)

target_state_pairs = [(target, state_var) for target in targets for state_var in state_variables]
target_env_pairs = [(name, target, env_targets) for target in targets for (name, env_targets) in baseline_reservoirs]
target_het_pairs = [(name, target, state_vars) for target in targets for (name, state_vars) in heterogeneous_reservoirs]


In [6]:
from pipeline_base import RCPipeline, Rescale, DaylightMask
from pipeline_hydroshoot import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)

def generate_single(target_state_pairs):
  for target, state_var in target_state_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': state_var, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=SingleReservoirGenerator(state_var=state_var, run_ids=run_ids, state_ids=state_ids),
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=False)
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def generate_multi(target_het_pairs):
  for name, target, state_vars in target_het_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': name, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=MultiReservoirGenerator(state_vars=state_vars, run_ids=run_ids, state_ids=state_ids),
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        GroupRescale(datasets=datasets, state_vars=state_vars, state_ids=state_ids)
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def generate_env(target_env_pairs):
  for name, target, env_targets in target_env_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': name, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=TargetReservoirGenerator(targets=env_targets, run_ids=run_ids),
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=True)
      ],
      **shared_pipeline_params,
    )
    yield pipeline

In [7]:
from itertools import chain

pipelines = list(chain(
  generate_single(target_state_pairs),
  generate_multi(target_het_pairs),
  generate_env(target_env_pairs)
))

print(f'Total pipelines: {len(pipelines)}')

Total pipelines: 112


### Execution

In [8]:
from tqdm import tqdm
from itertools import chain
from pipeline_base import execute_pipeline


RESULTS_FILE = 'results_hydroshoot_base_timemask_refactor.csv'

total_loops = len(pipelines)
results = []


with tqdm(total=total_loops) as pbar:
  for pipeline in pipelines:
    result = execute_pipeline(pipeline)
    results.append(result)
    pbar.update(1)


results_df = pd.DataFrame.from_dict(results)
results_df.to_csv(RESULTS_FILE)
print(f'Saved scores to {RESULTS_FILE}')

100%|██████████| 112/112 [00:34<00:00,  3.21it/s]

Saved scores to results_hydroshoot_base_timemask_refactor.csv
